In [1]:
using BenchmarkTools

In [2]:
include("src/staticexponents.jl")

IJulia.sp

In [3]:
x = sp.Variable{:x}()
y = sp.Variable{:y}()

y

In [5]:
@which x^2

^(x, p::Integer) in Base at intfuncs.jl:196

In [545]:
x^2 * y + y

1x^2y + 1y

In [546]:
typeof(x^2 * y + y + x * x + x * y * x + x)

IJulia.sp.Polynomial{Tuple{IJulia.sp.Term{x,Int64},IJulia.sp.Term{x^2,Int64},IJulia.sp.Term{x^2y,Int64},IJulia.sp.Term{y,Int64}}}

In [547]:
@code_warntype x^2

Variables:
  #self#::Base.#^
  x::IJulia.sp.Variable{:x}
  p::Int64

Body:
  begin 
      return $(Expr(:invoke, MethodInstance for power_by_squaring(::IJulia.sp.Variable{:x}, ::Int64), :(Base.power_by_squaring), :(x), :(p)))
  end::Any


In [529]:
@benchmark $x^2 + $y + $x * $x + 3 * $x * $y + $x * $y^2

BenchmarkTools.Trial: 
  memory estimate:  496 bytes
  allocs estimate:  20
  --------------
  minimum time:     48.342 μs (0.00% GC)
  median time:      50.207 μs (0.00% GC)
  mean time:        61.199 μs (0.00% GC)
  maximum time:     300.205 μs (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     1
  time tolerance:   5.00%
  memory tolerance: 1.00%

In [212]:
typeof(x + x)

IJulia.sp.Term{IJulia.sp.Monomial{((x, 1),)},Int64}

In [201]:
x + y

LoadError: [91mMethodError: no method matching +(::IJulia.sp.Polynomial{Tuple{IJulia.sp.Term{IJulia.sp.Monomial{((x, 1),)},Int64}}}, ::IJulia.sp.Polynomial{Tuple{IJulia.sp.Term{IJulia.sp.Monomial{((y, 1),)},Int64}}})[0m
Closest candidates are:
  +(::Any, ::Any, [91m::Any[39m, [91m::Any...[39m) at operators.jl:424[39m

In [179]:
sp.Term(x) :: sp.Term{<: sp.Monomial}

LoadError: [91mTypeError: typeassert: expected IJulia.sp.Term{#s2,T} where T where #s2<:IJulia.sp.Monomial, got IJulia.sp.Term{x^1,Int64}[39m

In [177]:
typeof(sp.Term(x)) <: Type{sp.Term{<:sp.Monomial, Int}}

false

In [167]:
@which sp.Term(x) + sp.Term(y)

+(t1::IJulia.sp.Term, t2::IJulia.sp.Term) in IJulia.sp at /Users/rdeits/.julia/v0.5/StaticPolynomials/src/staticexponents.jl:73

In [150]:
typeof(promote(x, y))

Tuple{IJulia.sp.Variable{:x},IJulia.sp.Variable{:y}}

In [151]:
x + y

LoadError: [91mMethodError: Cannot `convert` an object of type IJulia.sp.Variable{:x} to an object of type IJulia.sp.Term
This may have arisen from a call to the constructor IJulia.sp.Term(...),
since type constructors fall back to convert methods.[39m

In [130]:
x * x

x^2

In [131]:
x + x

2x^1

In [132]:
x * y

x^1y^1

In [135]:
1 * (x * y) + 3 * (x * y)

4x^1y^1

In [85]:
x^1 + x^1

2x^1

In [68]:
x * y

x^1y^1

In [ ]:
sp.Monomial{}

In [16]:
Base.promote(sp.Power(x), sp.Power(y))

LoadError: [91mUndefVarError: x not defined[39m

In [37]:
@code_warntype x * x

Variables:
  #self#::Base.#*
  v1::IJulia.sp.Variable{:x}
  v2::IJulia.sp.Variable{:x}

Body:
  begin 
      return $(QuoteNode(x^2))
  end::IJulia.sp.Power{x,2}


In [19]:
p = sp.Power{x, 2}()

x^2

In [401]:
module m

immutable Variable{Name}
end

immutable Power{V}
    exponent::Rational{Int}
end

end

IJulia.m

In [402]:
m.Power{m.Variable{:x}()}(1)

IJulia.m.Power{IJulia.m.Variable{:x}()}(1//1)

In [397]:
m.Power{m.Variable{:x}()}

LoadError: [91mTypeError: Power: in V, expected V<:IJulia.m.Variable, got IJulia.m.Variable{:x}[39m